In [6]:
import numpy as np
import pandas as pd
import sklearn.preprocessing
import sklearn.utils
import sklearn.metrics
import iisignature
import torch
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from typing import List, Optional, Dict, Set, Callable
from joblib import Memory, Parallel, delayed
import tslearn
import tslearn.metrics
from tslearn.datasets import UCR_UEA_datasets
import sigkernel
import scipy
from scipy.interpolate import interp1d

from signature import streams_to_sigs, transform_stream
from conformance import BaseclassConformanceScore, pairwise_kernel_gram, stream_to_torch

In [3]:
# Dataset: ArticularyWordRecognition
# Number of Classes: 25
# Dimension of path: 9
# Length: 144
# Train Size, Test Size 275 300

# Dataset: AtrialFibrillation
# No dataset found

# Dataset: BasicMotions
# Number of Classes: 4
# Dimension of path: 6
# Length: 100
# Train Size, Test Size 40 40

# Dataset: CharacterTrajectories
# No dataset found

# Dataset: Cricket
# Number of Classes: 12
# Dimension of path: 6
# Length: 1197
# Train Size, Test Size 108 72

# Dataset: DuckDuckGeese
# No dataset found

# Dataset: EigenWorms
# Number of Classes: 5
# Dimension of path: 6
# Length: 17984
# Train Size, Test Size 128 131

# Dataset: Epilepsy
# Number of Classes: 4
# Dimension of path: 3
# Length: 206
# Train Size, Test Size 137 138

# Dataset: EthanolConcentration
# Number of Classes: 4
# Dimension of path: 3
# Length: 1751
# Train Size, Test Size 261 263

# Dataset: ERing
# No dataset found

# Dataset: FaceDetection
# Number of Classes: 2
# Dimension of path: 144
# Length: 62
# Train Size, Test Size 5890 3524

# Dataset: FingerMovements
# Number of Classes: 2
# Dimension of path: 28
# Length: 50
# Train Size, Test Size 316 100

# Dataset: HandMovementDirection
# Number of Classes: 4
# Dimension of path: 10
# Length: 400
# Train Size, Test Size 160 74

# Dataset: Handwriting
# Number of Classes: 26
# Dimension of path: 3
# Length: 152
# Train Size, Test Size 150 850

# Dataset: Heartbeat
# Number of Classes: 2
# Dimension of path: 61
# Length: 405
# Train Size, Test Size 204 205

# Dataset: InsectWingbeat
# Number of Classes: 10
# Dimension of path: 200
# Length: 22
# Train Size, Test Size 25000 25000

# Dataset: JapaneseVowels
# No dataset found

# Dataset: Libras
# Number of Classes: 15
# Dimension of path: 2
# Length: 45
# Train Size, Test Size 180 180

# Dataset: LSST
# Number of Classes: 14
# Dimension of path: 6
# Length: 36
# Train Size, Test Size 2459 2466

# Dataset: MotorImagery
# Number of Classes: 2
# Dimension of path: 64
# Length: 3000
# Train Size, Test Size 278 100

# Dataset: NATOPS
# Number of Classes: 6
# Dimension of path: 24
# Length: 51
# Train Size, Test Size 180 180

# Dataset: PenDigits
# Number of Classes: 10
# Dimension of path: 2
# Length: 8
# Train Size, Test Size 7494 3498

# Dataset: PEMS-SF
# Number of Classes: 7
# Dimension of path: 963
# Length: 144
# Train Size, Test Size 267 173

# Dataset: Phoneme
# Number of Classes: 39
# Dimension of path: 1
# Length: 1024
# Train Size, Test Size 214 1896

# Dataset: RacketSports
# Number of Classes: 4
# Dimension of path: 6
# Length: 30
# Train Size, Test Size 151 152

# Dataset: SelfRegulationSCP1
# Number of Classes: 2
# Dimension of path: 6
# Length: 896
# Train Size, Test Size 268 293

# Dataset: SelfRegulationSCP2
# Number of Classes: 2
# Dimension of path: 7
# Length: 1152
# Train Size, Test Size 200 180

# Dataset: SpokenArabicDigits
# No dataset found

# Dataset: StandWalkJump
# Number of Classes: 3
# Dimension of path: 4
# Length: 2500
# Train Size, Test Size 12 15

# Dataset: UWaveGestureLibrary
# Number of Classes: 8
# Dimension of path: 3
# Length: 315
# Train Size, Test Size 120 320


# tslearn datasets (equal length)

* equal length (in time) UCR_UEA multivariate time series 

In [33]:
##########################################################################
######################## Static Kernels on R^d ###########################
##########################################################################

def _check_gram_dims(X:np.ndarray, 
                     Y:np.ndarray,
                     diag:bool = False,):
    """Stacks the input into a Gram matrix shape (N1, N2, ..., d) or
    into a diagonal Gram shape (N1, ..., d) if diag and N1==N2.

    Args:
        X (np.ndarray): Shape (N1, ... , d).
        Y (np.ndarray): Shape (N2, ... , d).
        diag (bool): If True, use diagonal Gram shape.
    """
    if len(X.shape)<2 or len(Y.shape)<2:
        raise ValueError("X and Y must have at least 2 dimensions.")
    if X.shape[1:] != Y.shape[1:]:
        raise ValueError("X and Y must have the same dimensions except for the first axis.")

    N1 = X.shape[0]
    N2 = Y.shape[0]
    if diag and N1!=N2:
        raise ValueError("If 'diag' is True, X and Y must have the same number of samples.")


def linear_kernel_gram(X:np.ndarray, 
                       Y:np.ndarray,
                       diag:bool = False,
                       ):
    """Computes the Rd inner product matrix <x_i, y_j> or diagonal <x_i, y_i>.
    The inputs dimensions can only differ in the first axis.
    
    Args:
        X (np.ndarray): Shape (N1, ... , d).
        Y (np.ndarray): Shape (N2, ... , d).
        diag (bool): If True, computes the diagonal of the gram matrix.

    Returns:
        np.ndarray: Array of shape (N1, N2, ...) or (N1, ...) if diag=True.
    """
    _check_gram_dims(X, Y, diag)
    if diag:
        #out_i... = sum(X_i...k * Y_i...k)
        return np.einsum('i...k,i...k -> i...', X, Y)
    else:
        #out_ij... = sum(X_i...k * Y_j...k)
        return np.einsum('i...k,j...k -> ij...', X, Y)


def rbf_kernel_gram(X:np.ndarray, 
                    Y:np.ndarray,
                    gamma:float,
                    diag:bool = False,
                    ):
    """Computes the RBF gram matrix k(x_i, y_j) or diagonal k(x_i, y_i).
    The inputs dimensions can only differ in the first axis.
    
    Args:
        X (np.ndarray): Shape (N1, ... , d).
        Y (np.ndarray): Shape (N2, ... , d).
        gamma (float): RBF parameter
        diag (bool): If True, computes the diagonal of the gram matrix.

    Returns:
        np.ndarray: Array of shape (N1, N2, ...) or (N1, ...) if diag=True.
    """
    if diag:
        diff = X-Y
        norms_squared = linear_kernel_gram(diff, diff, diag=True)
    else:
        xx = linear_kernel_gram(X, X, diag=True)
        xy = linear_kernel_gram(X, Y, diag=False)
        yy = linear_kernel_gram(Y, Y, diag=True)
        norms_squared = xx[:, np.newaxis] -2*xy + yy[np.newaxis, :]

    d= X.shape[-1]
    return np.exp(-gamma * norms_squared/d)


def poly_kernel_gram(X:np.ndarray, 
                     Y:np.ndarray,
                     p:float, #eg 2 or 3
                     diag:bool = False):
    """Computes the polynomial kernel (<x_i, y_j> + 1)^p.
    The inputs dimensions can only differ in the first axis.
    
    Args:
        X (np.ndarray): Shape (N1, ... , d).
        Y (np.ndarray): Shape (N2, ... , d).
        p (float): Polynomial degree.
        diag (bool): If True, computes the diagonal of the gram matrix.

    Returns:
        np.ndarray: Array of shape (N1, N2, ...) or (N1, ...) if diag=True.
    """
    d = X.shape[-1]
    xy = linear_kernel_gram(X, Y, diag)
    return (xy + d)**p


#######################################################################################
################### time series Integral Kernel of static kernel ######################
#######################################################################################


def integral_kernel(static_diag_kernel:Callable,
                    s1: np.ndarray,
                    s2: np.ndarray
                    )-> float:
    """Computes the integral kernel K(x, y) = \int k(x_t, y_t) dt 
    given static kernel and two piecewise linear paths.

    Args:
        static_diag_kernel_gram (Callable): Takes in two arrays of shape (M, d) 
                        and outputs the diagonal Gram <x_m, y_m> of shape (M).
        s1 (np.ndarray): A time series of shape (T1, d)
        s2 (np.ndarray): A time series of shape (T2, d)
    """
    #Find all breakpoints of the piecewise linear paths
    T1, d = s1.shape
    T2, d = s2.shape
    times = np.concatenate([np.linspace(0, 1, T1), np.linspace(0, 1, T2)])
    times = sorted(np.unique(times))

    #Add the extra breakpoints to the paths
    f1 = interp1d(np.linspace(0, 1, T1), s1, axis=0, assume_sorted=True)
    f2 = interp1d(np.linspace(0, 1, T2), s2, axis=0, assume_sorted=True)
    x = f1(times) #shape (len(times), d)
    y = f2(times)

    #calculate k(x_t, y_t) for each t
    Kt = static_diag_kernel(x, y)

    #return integral of k(x_t, y_t) dt
    return np.trapz(Kt, times)


def integral_kernel_gram(
        static_kernel_gram:Callable, #either linear_kernel_gram or rbf_kernel_gram with "diag" argument
        X:List[np.ndarray],
        Y:List[np.ndarray],
        variable_length:bool,
    ):
    """Computes the Gram matrix K(X_i, Y_j) of the integral kernel 
    K(x, y) = \int k(x_t, y_t) dt.


    Args:
        static_kernel_gram (Callable): Gram kernel function taking in two ndarrays and
                    one boolean "diag" argument, see e.g. 'linear_kernel_gram' or 
                    'rbf_kernel_gram'.
        X (List[np.ndarray]): List of time series of shape (T_i, d).
        Y (List[np.ndarray]): List of time series of shape (T_i, d).
        variable_length (bool): If False, uses the optimized kernels for equal 
                                length time series.
    """
    if not variable_length:
        X = np.array(X)
        Y = np.array(Y)
        ijKt = static_kernel_gram(X, Y, False) #diag=False

        #return integral of k(x_t, y_t) dt for each pair x and y
        N, T, d = X.shape
        return np.trapz(ijKt, dx=1/T, axis=-1)
    else:
        # static kernel with diag=True
        static_ker = lambda a,b : static_kernel_gram(a,b, True) #diag=True
        pairwise_int_ker = lambda s1, s2 : integral_kernel(static_ker, s1, s2)
        return pairwise_kernel_gram(X,
                                    Y,
                                    pairwise_int_ker,
                                    sym=True)

###########################################################################
########################## signature kernels ##############################
###########################################################################

In [25]:
def print_dataset_stats(num_classes, d, T, N_train, N_test):
    print("Number of Classes:", num_classes)
    print("Dimension of path:", d)
    print("Length:", T)
    print("Train:", N_train)
    print("Test:", N_test)


def case_static(train:np.ndarray, 
                test:np.ndarray,
                static_kernel_gram:Callable):
    """Calculates the gram matrices of equal length time series for 
    a static kernel on R^d. Train and test are of shape (N1, T, d) 
    and (N2, T, d). Static kernel should take in two arrays of shape 
    (M, T*d) and return the Gram matrix."""
    N, T = train.shape[:2]
    train = train.reshape(N, -1)
    test = test.reshape(N, -1)
    vv_gram = static_kernel_gram(train, train)/T
    uv_gram = static_kernel_gram(test, train)/T
    return vv_gram, uv_gram


def case_linear(train:np.ndarray, 
                test:np.ndarray):
    """Calculates the gram matrices for the euclidean inner product.
    Train and test are of shape (N1, T, d) and (N2, T, d)."""
    return case_static(train, test, linear_kernel_gram)


def case_rbf(train:np.ndarray, 
                   test:np.ndarray,
                   gamma:float):
    """Calculates the gram matrices for the rbf kernel.
    Train and test are of shape (N1, T, d) and (N2, T, d)."""
    rbf_ker = lambda X, Y : rbf_kernel_gram(X, Y, gamma)
    return case_static(train, test, rbf_ker)


def case_poly(train:np.ndarray, 
            test:np.ndarray,
            p:float):
    """Calculates the gram matrices for the rbf kernel.
    Train and test are of shape (N1, T, d) and (N2, T, d)."""
    poly_ker = lambda X, Y : poly_kernel_gram(X, Y, p)
    return case_static(train, test, poly_ker)


def case_gak(train:List[np.ndarray], 
                   test:List[np.ndarray], 
                   variable_length:bool,
                   sigma:float = 1.0,):
    """Calculates the gram matrices for the gak kernel.
    Train and test are lists of possibly variable length multidimension 
    time series of shape (T_i, d)"""
    #pick sigma parameter according to GAK paper
    if not variable_length:
        sigma = tslearn.metrics.sigma_gak(np.array(train))

    #compute gram matrices
    kernel = lambda s1, s2 : tslearn.metrics.gak(s1, s2, sigma)
    vv_gram = pairwise_kernel_gram(train, train, kernel, sym=True, disable_tqdm=False)
    uv_gram = pairwise_kernel_gram(test, train, kernel, sym=False, disable_tqdm=False)
    return vv_gram, uv_gram



#TODO TODO create new truncsig with custom kernel
def choose_trunc_order(d:int, trunc_sig_dim_bound:int=1000):
    """Chooses the order N such that the truncated signature has dimensionality
    less than some bound. The dimensionality is d**(N+1) - d"""
    N = -1 + np.log(d+trunc_sig_dim_bound)/np.log(d)
    N = max(2, np.floor(N))
    return int(N)
def case_trunc_sig(train:List[np.ndarray], 
                   test:List[np.ndarray], 
                   variable_length:bool,
                   trunc_sig_dim_bound:int,
                   ):
    """Calculates the gram matrices for the truncated signature.
    Train and test are lists of possibly variable length multidimension 
    time series of shape (T_i, d)"""
    d = train[0].shape[1]
    order = choose_trunc_order(d, trunc_sig_dim_bound)
    if variable_length:
        train = streams_to_sigs(train, order, disable_tqdm=False)
        test = streams_to_sigs(test, order, disable_tqdm=False)
    else: 
        train = iisignature.sig(np.array(train), order)
        test = iisignature.sig(np.array(test), order)
    return case_linear(train, test)


def case_sig_pde(train:List[np.ndarray], 
                 test:List[np.ndarray], 
                 dyadic_order:int = 3,
                 static_kernel = sigkernel.LinearKernel(),
                ):
    """Calculates the signature kernel gram matrices of the train and test.
    Train and test are lists of possibly variable length multidimension 
    time series of shape (T_i, d)"""
    sig_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)
    kernel = lambda s1, s2 : sig_kernel.compute_kernel(
                                stream_to_torch(s1), 
                                stream_to_torch(s2)).numpy()[0]
    vv_gram = pairwise_kernel_gram(train, train, kernel, sym=True, disable_tqdm=False)
    uv_gram = pairwise_kernel_gram(test, train, kernel, sym=False, disable_tqdm=False)
    return vv_gram, uv_gram


def calc_grams(train:List[np.ndarray], 
               test:List[np.ndarray],
               kernel_name:str, 
               variable_length:bool, 
               dyadic_order:int,        #for signature pde
               trunc_sig_dim_bound:int,  #for truncated signature
               gamma:float = 2,       #for rbf
               p:float = 2,             #for polynomial
               ):   
    """Calculates gram matrices <train, train>, <test, train> given a kernel.
    Train and test are lists of possibly variable length multidimension time 
    series of shape (T_i, d)"""

    #Transform to array if possible
    if not variable_length:
        train = np.array(train)
        test = np.array(test)
    
    #choose method based on kernel name
    if kernel_name == "linear":
        return case_linear(train, test)
    
    elif kernel_name == "rbf":
        return case_rbf(train, test, gamma)
    
    elif kernel_name == "poly":
        return case_poly(train, test, p)

    elif kernel_name == "gak":
        return case_gak(train, test, variable_length)

    elif kernel_name == "truncated signature":
        return case_trunc_sig(train, test, variable_length,
                                    trunc_sig_dim_bound)
    
    elif kernel_name == "signature pde":
        return case_sig_pde(train, 
                        test, 
                        variable_length,
                        dyadic_order=dyadic_order, 
                        static_kernel=sigkernel.LinearKernel(),)
    
    elif kernel_name == "signature pde RBF":
        return case_sig_pde(train, 
                        test, 
                        variable_length,
                        dyadic_order=dyadic_order, 
                        static_kernel=sigkernel.RBFKernel(sigma=0.5),)
    
    elif kernel_name == "integral linear":
        vv_gram = integral_kernel_gram(linear_kernel_gram, train, train, variable_length)
        uv_gram = integral_kernel_gram(linear_kernel_gram, test, train, variable_length)
        return vv_gram, uv_gram

    elif kernel_name == "integral rbf":
        ker = lambda X, Y, diag: rbf_kernel_gram(X, Y, gamma, diag)
        vv_gram = integral_kernel_gram(ker, train, train, variable_length)
        uv_gram = integral_kernel_gram(ker, test, train, variable_length)
        return vv_gram, uv_gram

    elif kernel_name == "integral poly":
        ker = lambda X, Y, diag : poly_kernel_gram(X, Y, p, diag)
        vv_gram = integral_kernel_gram(ker, train, train, variable_length)
        uv_gram = integral_kernel_gram(ker, test, train, variable_length)
        return vv_gram, uv_gram
    
    else:
        raise ValueError("Invalid kernel name:", kernel_name)


def normalize_streams(train:np.ndarray, 
                      test:np.ndarray,
                      ):
    """Inputs are 3D arrays of shape (N, T, d) where N is the number of time series, 
    T is the length of each time series, and d is the dimension of each time series."""
    # Normalize data by training set mean and std
    mean = np.mean(train, axis=0, keepdims=True)
    std = np.std(train, axis=0, keepdims=True)
    train = (train - mean) / std
    test = (test - mean) / std
    return train, test


def run_single_kernel(X_train:List[np.ndarray], 
                    y_train:np.array, 
                    X_test:List[np.ndarray], 
                    y_test:np.array, 
                    labels:np.array, 
                    kernel_name:str,
                    variable_length:bool,
                    normalize:bool,
                    dyadic_order:int = 3,   #for signature pde
                    trunc_sig_dim_bound:int = 1000, #for truncated signature
                    SVD_threshold:float = 0.01,
                    SVD_max_rank:Optional[int] = None,
                    gamma:float = 2,       #for rbf
                    ):
    """Computes the AUC scores (weighted one vs rest) for a single kernel,
    using kernelized nearest neighbour variance adjusted distances.

    Args:
        X_train (List[np.ndarray]): List of time series of shape (T_i, d).
        y_train (np.array): 1-dim array of class labels.
        X_test (List[np.ndarray]): List of time series of shape (T_i, d).
        y_test (np.array): 1-dim array of class labels.
        labels (np.array): Array of unique class labels.
        kernel_name (str): Name of the kernel to use.
        variable_length (bool): If False, uses the optimized kernels for equal 
                                length time series.
        normalize (bool): If True, normalizes train and test by the training set
                          mean and std.
        dyadic_order (int): Dyadic order for PDE solver 
                            (int > 0, higher = more accurate but slower).
        max_batch (int): Batch size in sig kernel computations.
        trunc_sig_dim_bound (int): Upper bound on the dimensionality of the 
                                  truncated signature.
        SVD_threshold (float): Sets all eigenvalues below this threshold to be 0.
        SVD_max_rank (int): Sets all SVD eigenvalues to be 0 beyond 'SVD_max_rank'.
    """
    # 2 methods (conf, mahal), 2 metrics (roc_auc, pr_auc), C classes
    C = len(labels)
    aucs = np.zeros( (2, 2, C) ) 

    for i, label in enumerate(labels):
        # Get all samples of the current class
        idxs = np.where(y_train == label)[0]
        corpus = [X_train[k] for k in idxs]
        test = X_test
        if normalize and not variable_length:
            corpus, test = normalize_streams(np.array(corpus), test)

        # Calculate amomaly distancce scores for all test samples
        vv_gram, uv_gram = calc_grams(corpus, test, kernel_name, 
                            variable_length, dyadic_order, 
                            trunc_sig_dim_bound, gamma=gamma)
        scorer = BaseclassConformanceScore(vv_gram, SVD_threshold, print_rank=True, 
                                           SVD_max_rank=SVD_max_rank)
        dists = np.array([scorer._anomaly_distance(sample, method="both") 
                          for sample in uv_gram]).T
        distances_conf, distances_mahal = dists

        # Calculate one vs rest AUC, weighted by size of class
        for idx_conf_mahal, distances in enumerate([distances_conf, distances_mahal]):
            ovr_labels = y_test != label
            average="weighted" #average = "macro"
            roc_auc = sklearn.metrics.roc_auc_score(ovr_labels, distances, average=average)
            pr_auc = sklearn.metrics.average_precision_score(ovr_labels, distances, average=average)
            aucs[idx_conf_mahal, 0, i] = roc_auc
            aucs[idx_conf_mahal, 1, i] = pr_auc
    
    return aucs


def run_tslearn_experiments(dataset_names:List[str], 
                            kernel_names:List[str],
                            gamma:float = 2,
                            ):
    """Runs a series of time series anomaly detection experiments on the specified 
    tslearn datasets using kernel conformance scores."""
    experiments = {}
    for dataset_name in dataset_names:
        # Load dataset
        X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset(dataset_name)

        # stats
        labels = np.unique(y_train)
        num_classes = len(labels)
        N_train, T, d = X_train.shape
        N_test, _, _  = X_test.shape
        print_dataset_stats(num_classes, d, T, N_train, N_test)

        # Run each kernel
        kernel_results = {}
        for kernel_name in kernel_names:
            print("Kernel:", kernel_name)
            scores = run_single_kernel(X_train, y_train, X_test, y_test, labels, 
                            kernel_name, SVD_threshold=0.001, SVD_max_rank=100,
                            variable_length=False, normalize=True, gamma=gamma)
            kernel_results[kernel_name] = scores
        
        #log dataset experiment
        experiments[dataset_name] = {"results": kernel_results, 
                                     "num_classes": num_classes, 
                                     "dim":d,
                                     "ts_length":T, 
                                     "N_train":N_train, 
                                     "N_test":N_test}
    return experiments

In [14]:
experiments = run_tslearn_experiments(
    dataset_names = [
        #'ArticularyWordRecognition', 
        #'BasicMotions', 
        #'Cricket',
         ##########'ERing', #cant find dataset
        'Libras', 
        #'NATOPS', 
        #'RacketSports',     
        #'FingerMovements',
        #'Heartbeat',
        #'SelfRegulationSCP1', 
        #'UWaveGestureLibrary'
        ],
    kernel_names = [
        #"linear", 
        "rbf",
        #"poly",
        # "gak",
        # "truncated signature",
        # "signature pde",
        # "signature pde RBF",
        #"integral linear",
        "integral rbf",
        #"integral poly",
        ],
        gamma=0.001)


def print_experiment_results(experiments, round_digits=5):
    for dataset_name, results in experiments.items():
        #Dataset:
        print("Dataset:", dataset_name)
        print_dataset_stats(results["num_classes"], results["dim"], 
                            results["ts_length"], results["N_train"], 
                            results["N_test"])


        #Results for each kernel:
        for kernel_name, scores in results["results"].items():
            print("\nKernel:", kernel_name)
            scores = np.mean(scores, axis=2)
            print("Conformance AUC:", round(scores[0, 0], round_digits))
            print("Mahalanobis AUC:", round(scores[1, 0], round_digits))
            print("Conformance PR AUC:", round(scores[0, 1], round_digits))
            print("Mahalanobis PR AUC:", round(scores[1, 1], round_digits))

        print("\nEnd Dataset\n\n\n")
        
print_experiment_results(experiments)

Number of Classes: 15
Dimension of path: 2
Length: 45
Train: 180
Test: 180
Kernel: rbf
Covariance operator numerical rank = 9
Covariance operator numerical rank = 8
Covariance operator numerical rank = 9
Covariance operator numerical rank = 10
Covariance operator numerical rank = 8
Covariance operator numerical rank = 6
Covariance operator numerical rank = 6
Covariance operator numerical rank = 9
Covariance operator numerical rank = 9
Covariance operator numerical rank = 7
Covariance operator numerical rank = 6
Covariance operator numerical rank = 8
Covariance operator numerical rank = 5
Covariance operator numerical rank = 6
Covariance operator numerical rank = 4
Kernel: integral rbf
Covariance operator numerical rank = 1
Covariance operator numerical rank = 1
Covariance operator numerical rank = 2
Covariance operator numerical rank = 1
Covariance operator numerical rank = 1
Covariance operator numerical rank = 2
Covariance operator numerical rank = 2
Covariance operator numerical ran

# PenDigits dataset (Variable Length) 

* Can't use ts-learn since it interpolated and homogenized the length of all time series

In [7]:
#################################################################################################################
## Loading code taken from https://github.com/pafoster/conformance_distance_experiments_cochrane_et_al_2020    ##
## DATASET_URLS = ['https://archive.ics.uci.edu/ml/machine-learning-databases/pendigits/pendigits-orig.tes.Z', ##
##                 'https://archive.ics.uci.edu/ml/machine-learning-databases/pendigits/pendigits-orig.tra.Z'] ##
#################################################################################################################

def read_pendigits_dataset(filename):
    with open(filename, 'r') as f:
        data_lines = f.readlines()

    data = []
    data_labels = []
    current_digit = None
    for line in data_lines:
        if line == "\n":
            continue

        if line[0] == ".":
            if "SEGMENT DIGIT" in line[1:]:
                if current_digit is not None:
                    data.append(np.array(current_digit))
                    data_labels.append(digit_label)

                current_digit = []
                digit_label = int(line.split('"')[1])
            else:
                continue

        else:
            x, y = map(float, line.split())
            current_digit.append([x, y])
            
    data.append(np.array(current_digit))
    data_labels.append(digit_label)
    return data, np.array(data_labels)


def create_pendigits_dataframe(data):
    dataframes = []
    for subset, data in data.items():
        df = pd.DataFrame(data).T
        df.columns = ['data', 'label']
        df['subset'] = subset
        dataframes.append(df)
    return pd.concat(dataframes)

data = {'train': read_pendigits_dataset("Data/pendigits-orig.tra"),
        'test': read_pendigits_dataset("Data/pendigits-orig.tes")}

df_pendigits_raw = create_pendigits_dataframe(data)

def plot_pendigits_entry(sample):
    df = pd.DataFrame(sample["data"], columns=["x", "y"])
    fig = px.line(df, x="x", y="y", text=df.index, width=500, height=500)
    fig.update_traces(textposition="bottom right")
    fig.show()

plot_pendigits_entry(df_pendigits_raw.iloc[20])
print(df_pendigits_raw.head()) 
# Each data point is a timeseries of the form ([x_t1, y_t1], ... , [x_tn, y_tn]) 
# of variable length, that is an array of shape (N_i, 2) for each i in the dataset.


FileNotFoundError: [Errno 2] No such file or directory: 'Data/pendigits-orig.tra'

In [ ]:
############################################################################################## |
################################### PenDigits experiments #################################### |
############################################################################################## \/

def run_pendigits_experiments(df:pd.DataFrame, 
                              kernel_names:List[str],
                              stream_transforms = ["time_enhance", "min_max_normalize"],):
    """Calculates AUCs for each kernel on the PenDigits dataset.
    df has columns ["data", "label", "subset"]. Each data point 
    is a timeseries of shape (T_i, d) of variable length."""
    #transform streams
    df["data"] = df["data"].apply(lambda x : transform_stream(x, stream_transforms))

    #Gather dataset info
    X_train = df[df["subset"]=="train"]["data"].values
    y_train = np.array(df[df["subset"]=="train"]["label"].values)
    X_test = df[df["subset"]=="test"]["data"].values
    y_test = np.array(df[df["subset"]=="test"]["label"].values)
    labels = sorted(df["label"].unique())
    num_classes = len(labels)
    d = X_train[0].shape[1]
    T = "variable length"
    N_train = len(X_train)
    N_test = len(X_test)
    print_dataset_stats(num_classes, d, T, N_train, N_test)

    # Run each kernel
    kernel_results = {}
    for kernel_name in kernel_names:
        print(kernel_name)
        scores = run_single_kernel(X_train, y_train, X_test, y_test, labels, 
                        kernel_name, variable_length=True, normalize=False,
                        trunc_sig_dim_bound=200, SVD_max_rank=None)
        kernel_results[kernel_name] = scores

    #log results
    pendigits_results = {"results": kernel_results, 
                         "num_classes": num_classes,
                         "dim": d,
                         "ts_length":T, 
                         "N_train":N_train, 
                         "N_test":N_test}
    return pendigits_results

# pendigits_results = run_pendigits_experiments(
#     df_pendigits_raw, 
#     kernel_names=[
#         #"gak",
#         "truncated signature", 
#         #"signature pde", 
#         #"signature pde RBF"
#         ],
#         )

NameError: name 'pd' is not defined

In [ ]:
# Dataset: Libras
# Number of Classes: 15
# Dimension of path: 2
# Length: 45
# Train: 180
# Test: 180

# Kernel: linear
# Conformance AUC: 0.9471891534391536
# Mahalanobis AUC: 0.9460978835978835
# Conformance PR AUC: 0.9952856063472626
# Mahalanobis PR AUC: 0.9953123143532209

# Kernel: rbf
# Conformance AUC: 0.6121858465608465
# Mahalanobis AUC: 0.235896164021164
# Conformance PR AUC: 0.9543784983385764
# Mahalanobis PR AUC: 0.9073331474510873

# Kernel: gak
# Conformance AUC: 0.8303240740740742
# Mahalanobis AUC: 0.056779100529100526
# Conformance PR AUC: 0.9720553520740843
# Mahalanobis PR AUC: 0.8271237492596946

# Kernel: truncated signature
# Conformance AUC: 0.8772486772486773
# Mahalanobis AUC: 0.8711970899470899
# Conformance PR AUC: 0.9884586491342986
# Mahalanobis PR AUC: 0.9880448383038575

# Kernel: signature pde
# Conformance AUC: 0.8616071428571429
# Mahalanobis AUC: 0.8559854497354498
# Conformance PR AUC: 0.9860276339146576
# Mahalanobis PR AUC: 0.9856341293618142

# Kernel: signature pde RBF
# Conformance AUC: 0.48647486772486775
# Mahalanobis AUC: 0.5124669312169311
# Conformance PR AUC: 0.9148945413486355
# Mahalanobis PR AUC: 0.9149847211152167

# End Dataset


In [ ]:
# print_experiment_results({"PenDigits": pendigits_results})

In [16]:
##############################################################
################ Playing around with Ksig ####################
##############################################################

import numpy as np
import ksig

# Number of signature levels to use.
n_levels = 5 

# Use the RBF kernel for vector-valued data as static (base) kernel.
static_kernel = ksig.static.kernels.RBFKernel() 

# Instantiate the signature kernel, which takes as input the static kernel.
sig_kernel = ksig.kernels.SignatureKernel(n_levels, static_kernel=static_kernel)

# Generate 10 sequences of length 50 with 5 channels.
n_seq, l_seq, n_feat = 2, 50, 5 
X = np.random.randn(n_seq, l_seq, n_feat)

# Sequence kernels take as input an array of sequences of ndim == 3,
# and work as a callable for computing the kernel matrix. 
K_XX = sig_kernel(X)  # K_XX has shape (10, 10).

# The diagonal kernel entries can also be computed.
K_X = sig_kernel(X, diag=True)  # K_X has shape (10,).

# Generate another array of 8 sequences of length 20 and 5 features.
n_seq2, l_seq2 = 8, 20
Y = np.random.randn(n_seq2, l_seq2, n_feat)

# Compute the kernel matrix between arrays X and Y.
K_XY = sig_kernel(X, Y)  # K_XY has shape (10, 8)

In [94]:
import time


def calc_iisig_kernel(X, Y, order):
    sig_X, sig_Y = streams_to_sigs([X,Y], order, disable_tqdm=True)
    print("sig_X", sig_X)
    print("sig_Y", sig_Y)
    dot = 1 + np.dot(sig_X, sig_Y)
    return dot


def calc_ksig_kernel(X, Y, order):
    static_kernel = ksig.static.kernels.LinearKernel() 
    sig_kernel = ksig.kernels.SignatureKernel(order, static_kernel=static_kernel)
    dot = sig_kernel(np.array([X,X]),Y[None,:])[0,0]
    return dot


d = 2
MAX_ORDER = 2
times_iisig = np.zeros( (MAX_ORDER) )
times_ksig  = np.zeros( (MAX_ORDER) )
for order in range(1, MAX_ORDER+1):
    X, Y = np.random.randn(2, 50, d)
    t0= time.time()
    sig1=calc_iisig_kernel(X, Y, order)
    t1 = time.time()
    sig2=calc_ksig_kernel(X, Y, order)
    t2 = time.time()
    times_iisig[order-1] = t1-t0
    times_ksig[order-1] = t2-t1
    print("sig1", sig1)
    print("sig2", sig2)
    print("sig1-sig2", sig1-sig2)


print("comparison", times_iisig/times_ksig)
print("\niisig", times_iisig)
print("\nksig", times_ksig)

sig_X [-2.4729541   0.30670244]
sig_Y [0.88073519 0.08890687]
sig1 -1.150749752128363
sig2 0.012491415965406483
sig1-sig2 -1.1632411680937693
sig_X [ 0.51075959 -0.45718983  0.13043768  4.90733483 -5.14084892  0.10451127]
sig_Y [-0.24567046  1.11724594  0.03017699  2.79655671 -3.07103103  0.62411924]
sig1 29.94423839293617
sig2 0.3518627096527344
sig1-sig2 29.592375683283436
comparison [0.36816186 0.08089722]

iisig [0.00147939 0.00049615]

ksig [0.00401831 0.00613308]


In [281]:
import numpy as np
from scipy.ndimage import shift



def calc_iisig_kernel(X, Y, order):
    sig_X, sig_Y = streams_to_sigs([X,Y], order, disable_tqdm=True)
    # print("sig_X", sig_X)
    # print("sig_Y", sig_Y)
    dot = 1 + np.dot(sig_X, sig_Y)
    return dot


# def old_calc_ksig_kernel(s1:np.ndarray, 
#                      s2:np.ndarray, 
#                      order:int):
#     """s1 and s2 are time series of shape (T_i, d)"""
#     K = linear_kernel_gram(s1, s2) #TODO add as argument
#     nabla = K[1:, 1:] + K[:-1, :-1] - K[1:, :-1] - K[:-1, 1:]
#     A = np.zeros((order, *nabla.shape))
#     A[0] = nabla
#     print("nabla", nabla)
#     for m in range(1, order):
#         #Reverse cumsums

#         Q = A[m-1]
#         # Q = np.cumsum(Q[::-1], axis=0)[::-1]
#         # Q = np.cumsum(Q.T[::-1], axis=0)[::-1].T
#         Q = np.cumsum(Q, axis=0)
#         Q = np.cumsum(Q, axis=1)
#         print("Q", Q)

#         shifted = (1+shift(Q, (0,0), cval=0))
#         A[m] = nabla * shifted
#         print("shifted", shifted)
#         # print("A[m]", A[m])
#         # print("nabla*shift", nabla * shifted)
#         # print("nabla again", nabla)
#         # print("matmul", nabla @ shifted)

#     return 1+np.sum(np.sum(A[order-1], axis=-1), axis=-1)



def calc_ksig_kernel(s1:np.ndarray, 
                     s2:np.ndarray, 
                     order:int):
    """s1 and s2 are time series of shape (T_i, d)"""
    K = linear_kernel_gram(s1, s2) #TODO add as argument
    nabla = K[1:, 1:] + K[:-1, :-1] - K[1:, :-1] - K[:-1, 1:]
    A = np.ones((order, *nabla.shape))
    for m in range(1, order):
        #obtain summands
        Q = nabla * A[m-1]

        # #Reverse cumsums
        # Q = np.cumsum(Q[::-1], axis=0)[::-1]
        # Q = np.cumsum(Q.T[::-1], axis=0)[::-1].T

        # #shift
        # A[m] = 1 + shift(Q, (-1,-1), cval=0)

        #cumsums
        Q = np.cumsum(Q, axis=0)
        Q = np.cumsum(Q, axis=1)

        #shift
        A[m] = 1 + shift(Q, (1,1), cval=0)

    return 1 + np.sum(nabla*A[-1])


d = 2
MAX_ORDER = 8
times_iisig = np.zeros( (MAX_ORDER) )
times_ksig  = np.zeros( (MAX_ORDER) )
for order in range(1, MAX_ORDER+1):
    print("order", order)
    np.random.seed(99)
    X, Y = np.random.randn(2, 5, d)
    t0= time.time()
    sig1=calc_iisig_kernel(X, Y, order)
    t1 = time.time()
    sig2=calc_ksig_kernel(X, Y, order)
    t2 = time.time()
    times_iisig[order-1] = t1-t0
    times_ksig[order-1] = t2-t1
    print("sig1", sig1)
    print("sig2", sig2)
    #print("sig1-sig2", sig1-sig2)

#TODO implement the other, less efficient PRODUCT FORMULA

# print("\ncomparison", times_iisig/times_ksig)
# print("\niisig", times_iisig)
# print("\nksig", times_ksig)

order 1
sig1 1.408125434785367
sig2 1.4081254347853687
order 2
sig1 0.21177077951716372
sig2 -10.735498526929668
order 3
sig1 -0.07772962768761205
sig2 -17.705741617783374
order 4
sig1 -0.10346728913889391
sig2 -19.81525818644514
order 5
sig1 0.2574217454957398
sig2 -19.81525818644513
order 6
sig1 0.3589555583778956
sig2 -19.815258186445142
order 7
sig1 0.33802976809676
sig2 -19.815258186445135
order 8
sig1 0.3387920971912588
sig2 -19.815258186445142


In [207]:
test = np.arange(3*3).reshape(3,3)
print(test)
test1 = np.cumsum(test[::-1], axis=0)[::-1]
print(test1, "test1")
test2 = np.cumsum(test1.T[::-1], axis=0)[::-1].T
print(test2, "test2")
print(np.flip(test2, axis=(-1, -2)), "flip")

print(shift(test2, (-1,-1), cval=0), "shift")

# [[0 1 2]
#  [3 4 5]
#  [6 7 8]]
# [[ 9 12 15]
#  [ 9 11 13]
#  [ 6  7  8]]
# [[36 27 15]
#  [33 24 13]
#  [21 15  8]]
# [[ 8 15 21]
#  [13 24 33]
#  [15 27 36]] flip
# [[24 13  0]
#  [15  8  0]
#  [ 0  0  0]] shift


[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[ 9 12 15]
 [ 9 11 13]
 [ 6  7  8]] test1
[[36 27 15]
 [33 24 13]
 [21 15  8]] test2
[[ 8 15 21]
 [13 24 33]
 [15 27 36]] flip
[[24 13  0]
 [15  8  0]
 [ 0  0  0]] shift


In [211]:
test = np.arange(3*3).reshape(3,3)
print(test)
test1 = np.cumsum(test[::-1], axis=0)[::-1]
test2 = np.cumsum(test1.T[::-1], axis=0)[::-1].T
print(test2,"test2")

print(shift(test2, (-1,-1), cval=0), "shift")


[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[36 27 15]
 [33 24 13]
 [21 15  8]] test2
[[24 13  0]
 [15  8  0]
 [ 0  0  0]] shift


In [4]:
import tslearn

_datasets = [
            'ArticularyWordRecognition', 
            'BasicMotions', 
            'Cricket',
            #'ERing',
            'Libras', 
            'NATOPS', 
            'RacketSports',     
            'FingerMovements',
            'Heartbeat',
            'SelfRegulationSCP1', 
            'UWaveGestureLibrary'
            ]


#for dataset_name in ucr_datasets.list_multivariate_datasets():
for dataset_name in _datasets:
    print("Dataset:", dataset_name)
    dataset = tslearn.datasets.UCR_UEA_datasets().load_dataset(dataset_name)
    if dataset[0] is not None:
        X_train, y_train, X_test, y_test = dataset
        num_classes = len(np.unique(y_train))
        N_train, T, d = X_train.shape
        N_test, _, _  = X_test.shape

        # is_irregular = dataset["is_irregular"]
        
        print("Number of Classes:", num_classes)
        print("Dimension of path:", d)
        print("Length:", T)
        print("Train Size, Test Size", N_train, N_test)
        print()
    else:
        print("No dataset found")
        print()

Dataset: ArticularyWordRecognition
Number of Classes: 25
Dimension of path: 9
Length: 144
Train Size, Test Size 275 300

Dataset: BasicMotions
Number of Classes: 4
Dimension of path: 6
Length: 100
Train Size, Test Size 40 40

Dataset: Cricket
Number of Classes: 12
Dimension of path: 6
Length: 1197
Train Size, Test Size 108 72

Dataset: Libras
Number of Classes: 15
Dimension of path: 2
Length: 45
Train Size, Test Size 180 180

Dataset: NATOPS
Number of Classes: 6
Dimension of path: 24
Length: 51
Train Size, Test Size 180 180

Dataset: RacketSports
Number of Classes: 4
Dimension of path: 6
Length: 30
Train Size, Test Size 151 152

Dataset: FingerMovements
Number of Classes: 2
Dimension of path: 28
Length: 50
Train Size, Test Size 316 100

Dataset: Heartbeat
Number of Classes: 2
Dimension of path: 61
Length: 405
Train Size, Test Size 204 205

Dataset: SelfRegulationSCP1
Number of Classes: 2
Dimension of path: 6
Length: 896
Train Size, Test Size 268 293

Dataset: UWaveGestureLibrary
Number

In [ ]:
# Dataset: ArticularyWordRecognition
# Number of Classes: 25
# Dimension of path: 9
# Length: 144
# Train Size, Test Size 275 300

# Dataset: BasicMotions
# Number of Classes: 4
# Dimension of path: 6
# Length: 100
# Train Size, Test Size 40 40

# Dataset: Cricket
# Number of Classes: 12
# Dimension of path: 6
# Length: 1197
# Train Size, Test Size 108 72

# Dataset: Libras
# Number of Classes: 15
# Dimension of path: 2
# Length: 45
# Train Size, Test Size 180 180

# Dataset: NATOPS
# Number of Classes: 6
# Dimension of path: 24
# Length: 51
# Train Size, Test Size 180 180

# Dataset: RacketSports
# Number of Classes: 4
# Dimension of path: 6
# Length: 30
# Train Size, Test Size 151 152

# Dataset: FingerMovements
# Number of Classes: 2
# Dimension of path: 28
# Length: 50
# Train Size, Test Size 316 100

# Dataset: Heartbeat
# Number of Classes: 2
# Dimension of path: 61
# Length: 405
# Train Size, Test Size 204 205

# Dataset: SelfRegulationSCP1
# Number of Classes: 2
# Dimension of path: 6
# Length: 896
# Train Size, Test Size 268 293

# Dataset: UWaveGestureLibrary
# Number of Classes: 8
# Dimension of path: 3
# Length: 315
# Train Size, Test Size 120 320